# 🌡️ AI-Driven Urban Heat Island (UHI) Analysis and Mitigation using Google Earth Engine + Gemini

## 🚧 Problem
Urban Heat Islands (UHIs) are localized zones in cities with significantly higher temperatures due to excessive concrete, low vegetation, and high human activity. These zones contribute to energy overuse, poor air quality, and health risks.

## 🎯 Goal
We aim to identify heat-prone zones along the U.S. West Coast using satellite-based thermal imagery and provide region-specific cooling strategies powered by Google Gemini (LLM). Our pipeline includes:

- Using Google Earth Engine (GEE) to detect UHI zones
- Visualizing thermal data and heat zones interactively
- Generating actionable sustainability recommendations using Gemini


## 🔧 1. Authenticate and Initialize Earth Engine

This section authenticates your Google Earth Engine account and initializes the environment. Required once per runtime session.


In [4]:
import ee

# Trigger authentication flow
ee.Authenticate()

# Initialize Earth Engine with your project ID
ee.Initialize(project='prime-phalanx-463704-u4')


## 🔥 4. Detect Urban Heat Islands with Landsat 8

This section:
- Loads West Coast US states
- Filters Landsat 8 imagery for low-cloud, summer data
- Computes land surface temperature (LST)
- Extracts and vectorizes regions with LST > 310K (~37°C)
- Saves output as GeoJSON

In [5]:
import geemap
import json

# Load West Coast states
states = ee.FeatureCollection("TIGER/2018/States")
west_coast = states.filter(ee.Filter.inList('NAME', ['California', 'Oregon', 'Washington']))

# Load and filter Landsat 8 imagery
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2022-06-01', '2022-08-31') \
    .filterBounds(west_coast) \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .median()

# Convert thermal band to LST in Kelvin
thermal = landsat.select('ST_B10').multiply(0.00341802).add(149)

# Create a binary heat mask for pixels > 310K
heat_mask = thermal.gt(310)

# Convert heat zones to polygons
heat_vectors = heat_mask.selfMask().reduceToVectors(
    geometry=west_coast.geometry(),
    geometryType='polygon',
    scale=1000,
    maxPixels=1e8
)

# Export as GeoJSON
geojson = geemap.ee_to_geojson(heat_vectors)
with open("west_coast_heat_zones.json", "w") as f:
    json.dump(geojson, f)

print("✅ Heat zones saved to west_coast_heat_zones.json")


✅ Heat zones saved to west_coast_heat_zones.json


## 🗺️ 5. Visualize Surface Temperature and Heat Zones

This cell renders an interactive map showing:
- LST imagery as a background
- Highlighted heat-prone zones (temperature > 37°C)


In [6]:
m = geemap.Map()
m.centerObject(west_coast, zoom=5)

vis_params = {
    'min': 290,
    'max': 320,
    'palette': ['blue', 'lime', 'yellow', 'orange', 'red']
}

m.add_ee_layer(thermal, vis_params, "LST - Surface Temp")
m.add_ee_layer(heat_vectors.style(**{'color': 'darkred', 'fillColor': 'FF000080'}), {}, "Heat Zones > 37°C")
m


Map(center=[41.34849667120548, -120.04086128111338], controls=(WidgetControl(options=['position', 'transparent…

## 🤖 6. Load Heat Zones & Connect to Gemini AI

This step:
- Loads the GeoJSON heat zones (limit to 400 features for token safety)
- Authenticates and prepares Gemini model for recommendations


In [7]:
import google.generativeai as genai
from google.colab import userdata

# Configure Gemini API key (must be set via Colab's secrets tool)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash-latest")

try:
    with open("west_coast_heat_zones.json", "r") as f:
        heat_island_data = json.load(f)
    print("✅ Heat zone data loaded from west_coast_heat_zones.json")
except FileNotFoundError:
    print("⚠️ File not found. Please generate heat zone data first.")
    heat_island_data = None

# Limit to 400 features
if heat_island_data and 'features' in heat_island_data:
    heat_island_data['features'] = heat_island_data['features'][:400]
    print(f"Limited heat zone features to {len(heat_island_data['features'])}")


✅ Heat zone data loaded from west_coast_heat_zones.json
Limited heat zone features to 400


## 🧠 7. Generate Mitigation Strategies with Gemini

Send the GeoJSON heat zone data to Gemini for:
1. Region-specific cooling actions (e.g., green roofs, tree planting)
2. Overall urban strategies for the West Coast
3. Estimated impact of recommendations (°C reduction, NDVI increase, etc.)


In [9]:
if heat_island_data: # Only construct prompt if data was loaded
    # Limit to 400 features
    if 'features' in heat_island_data:
        heat_island_data['features'] = heat_island_data['features'][:400]
        print(f"Limited heat zone features to {len(heat_island_data['features'])}")

    prompt = f"""
You are an expert in sustainable urban planning and climate adaptation.

Given the following GeoJSON data for urban regions affected by heat waves:

{json.dumps(heat_island_data, indent=2)}

Please analyze the provided GeoJSON data which represents heat zones on the West Coast.
Focus on the spatial distribution and characteristics of these zones to provide:

1. Region-specific recommendations to reduce heat, considering vegetation levels, urban density, and local temperature — include nature-based solutions like tree canopy enhancement, cool roofs, green corridors, reflective surfaces, or urban design strategies.

2. Broader strategic recommendations for cities on the U.S. West Coast to mitigate Urban Heat Island (UHI) effects using the trends observed in this data — with examples from other cities if relevant.

3. For each regional and overall recommendation, provide estimated impacts (qualitative or quantitative), such as expected temperature reduction, increase in NDVI, or reduced energy consumption.

"""

    # Step 5: Call Gemini
    response = model.generate_content(prompt)

    # Step 6: Output
    print("\n🧠 Gemini's Recommendations:\n")
    print(response.text)
else:
    print("Skipping Gemini call due to missing heat zone data.")

Limited heat zone features to 400

🧠 Gemini's Recommendations:

The provided GeoJSON data shows a cluster of heat zones along the West Coast, though the specific location isn't explicitly defined by latitude and longitude alone.  The `count` property suggests the intensity or frequency of heat events within each polygon.  A more robust analysis would require additional data layers (e.g., land cover, population density, building materials) to accurately assess the contributing factors and model the effectiveness of interventions. However, based on the available data and general knowledge of West Coast urban areas, we can offer preliminary recommendations.


**1. Region-Specific Recommendations (Hypothetical Regions):**

To offer region-specific recommendations, we will assume the data represents three hypothetical regions based on latitude clustering: Southern California (lower latitudes), Central Coast (mid-latitudes), and Northern California/Pacific Northwest (higher latitudes).  Thes

In [12]:
import google.generativeai as genai
from google.colab import userdata
import json

# Configure Gemini API key (must be set via Colab's secrets tool)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash-latest")

try:
    with open("west_coast_heat_zones.json", "r") as f:
        heat_island_data = json.load(f)
    print("✅ Heat zone data loaded from west_coast_heat_zones.json")
except FileNotFoundError:
    print("⚠️ File not found. Please generate heat zone data first.")
    heat_island_data = None

# Limit to 400 features
if heat_island_data and 'features' in heat_island_data:
    heat_island_data['features'] = heat_island_data['features'][:400]
    print(f"Limited heat zone features to {len(heat_island_data['features'])}")

if heat_island_data:  # Only construct prompt if data was loaded
    prompt = f"""
You are an expert in sustainable urban planning and climate adaptation.

Given the following GeoJSON data for urban regions affected by heat waves:

{json.dumps(heat_island_data, indent=2)}

Please analyze the provided GeoJSON data representing heat zones on the U.S. West Coast.

Based on the spatial distribution and characteristics of these zones, provide the following:

1. A Markdown-formatted table of region-specific recommendations using the format below:

| Region Name | Heat Zone Description | Recommendation | Estimated Impact |

- "Region Name" should be the GeoJSON feature ID or a short identifier if available.
- "Heat Zone Description" should summarize spatial or thermal characteristics of the region.
- "Recommendation" should be actionable (e.g., plant trees, install green roofs, reflective surfaces, etc.).
- "Estimated Impact" should be qualitative or quantitative (e.g., ~1.5°C reduction, +0.1 NDVI, etc.).

2. Strategic recommendations for mitigating Urban Heat Island (UHI) effects across West Coast cities, based on trends observed in the data. Reference successful interventions from other cities if relevant.

3. A summary of expected citywide or regional impacts if the above recommendations are implemented.
"""

    # Step 5: Call Gemini
    response = model.generate_content(prompt)

    # Step 6: Output
    print("\n🧠 Gemini's Recommendations:\n")
    print(response.text)
else:
    print("Skipping Gemini call due to missing heat zone data.")

✅ Heat zone data loaded from west_coast_heat_zones.json
Limited heat zone features to 400

🧠 Gemini's Recommendations:

The provided GeoJSON data lacks sufficient information to accurately describe the thermal characteristics of each heat zone (e.g., actual temperature readings, surface material types).  The "count" property might represent the number of heat wave events recorded in the area, but without further context, a precise heat zone description isn't possible.  Therefore, recommendations and impact assessments will be generalized based on the assumption that higher "count" values indicate more intense or frequent heat.


**1. Region-Specific Recommendations (Generalized)**

| Region Name      | Heat Zone Description                               | Recommendation                                                                  | Estimated Impact                               |
|-----------------|-------------------------------------------------------|----------------------------